In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = "Simple_GENAI_APP"
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['USER_AGENT'] = 'myagent'


In [14]:
# Data Ingestion

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://medium.com/@asimsultan2/understanding-transformers-a-simplified-guide-with-easy-to-understand-examples-227fd8d31848")
docs = loader.load()
docs

[Document(metadata={'source': 'https://medium.com/@asimsultan2/understanding-transformers-a-simplified-guide-with-easy-to-understand-examples-227fd8d31848', 'title': 'Understanding Transformers: A Simplified Guide with Easy-to-Understand Examples | by Asimsultan (Head of AI) | Medium', 'description': 'The Transformer architecture is a powerful model in natural language processing (NLP) that has revolutionized how machines understand and generate human language. However, the concepts behind…', 'language': 'en'}, page_content='Understanding Transformers: A Simplified Guide with Easy-to-Understand Examples | by Asimsultan (Head of AI) | MediumOpen in appSign upSign inWriteSign upSign inUnderstanding Transformers: A Simplified Guide with Easy-to-Understand ExamplesAsimsultan (Head of AI)·Follow4 min read·Aug 29, 2024--ListenShareThe Transformer architecture is a powerful model in natural language processing (NLP) that has revolutionized how machines understand and generate human language. 

In [17]:
# split document into small document chunk

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)
split_docs

[Document(metadata={'source': 'https://medium.com/@asimsultan2/understanding-transformers-a-simplified-guide-with-easy-to-understand-examples-227fd8d31848', 'title': 'Understanding Transformers: A Simplified Guide with Easy-to-Understand Examples | by Asimsultan (Head of AI) | Medium', 'description': 'The Transformer architecture is a powerful model in natural language processing (NLP) that has revolutionized how machines understand and generate human language. However, the concepts behind…', 'language': 'en'}, page_content='Understanding Transformers: A Simplified Guide with Easy-to-Understand Examples | by Asimsultan (Head of AI) | MediumOpen in appSign upSign inWriteSign upSign inUnderstanding Transformers: A Simplified Guide with Easy-to-Understand ExamplesAsimsultan (Head of AI)·Follow4 min read·Aug 29, 2024--ListenShareThe Transformer architecture is a powerful model in natural language processing (NLP) that has revolutionized how machines understand and generate human language. 

In [18]:
len(split_docs)

14

In [20]:
# create embedding

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000028951919900>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000028953449D80>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [21]:
from langchain_community.vectorstores import FAISS

vectorstoredb = FAISS.from_documents(split_docs, embeddings)
vectorstoredb

In [22]:
vectorstoredb.save_local("./simple_genai_faiss")

In [23]:
# similarity search

result_responses = vectorstoredb.similarity_search(query="let’s dive into multi-head attention")
result_responses

[Document(metadata={'source': 'https://medium.com/@asimsultan2/understanding-transformers-a-simplified-guide-with-easy-to-understand-examples-227fd8d31848', 'title': 'Understanding Transformers: A Simplified Guide with Easy-to-Understand Examples | by Asimsultan (Head of AI) | Medium', 'description': 'The Transformer architecture is a powerful model in natural language processing (NLP) that has revolutionized how machines understand and generate human language. However, the concepts behind…', 'language': 'en'}, page_content='combining their findings, you get a more complete understanding of the case. Similarly, each head in the multi-head attention mechanism looks at different relationships between words in a sentence. One head might focus on the subject-verb relationship (like “cat” and “chased”), while another head might focus on the object (like “mouse”).For example, in the sentence:“The quick brown fox jumps over the lazy dog.”One attention head might focus on how “quick” describes

In [24]:
result_responses[0].page_content

'combining their findings, you get a more complete understanding of the case. Similarly, each head in the multi-head attention mechanism looks at different relationships between words in a sentence. One head might focus on the subject-verb relationship (like “cat” and “chased”), while another head might focus on the object (like “mouse”).For example, in the sentence:“The quick brown fox jumps over the lazy dog.”One attention head might focus on how “quick” describes “fox,” while another might'

Create prompt template

In [25]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following questions based only on provided context:
    <context>
    {context} 
    </context>

"""
)

document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following questions based only on provided context:\n    <context>\n    {context} \n    </context>\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002897E89F5B0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002897E8C0BE0>, root_client=<openai.OpenAI object at 0x000002895341FB50>, root_async_client=<openai.AsyncOpenAI object at 0x000002897E89F610>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name

In [27]:
from langchain_core.documents import Document

document_chain.invoke(
    {
        "input": "let’s dive into multi-head attention",
        "context": [Document(page_content="let’s dive into multi-head attention. Think of it as getting multiple perspectives on the same situation. Each “head” in the multi-head attention mechanism looks at different aspects of the sentence.")]
    }
)

'Sure, here are some answers based on the provided context:\n\n1. **What is multi-head attention?**\n   - Multi-head attention is a mechanism that provides multiple perspectives on the same situation by using multiple "heads" to examine different aspects of a sentence.\n\n2. **What does each “head” in multi-head attention do?**\n   - Each "head" in the multi-head attention mechanism looks at different aspects of the sentence, providing diverse insights or perspectives.'

In [28]:
# retriever
retriever = vectorstoredb.as_retriever()

from langchain.chains import create_retrieval_chain

retriever_chain = create_retrieval_chain(retriever, document_chain)
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000028969D1C9A0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following questions based only on provided context:\n    <context>\n    {context} \n    </context>\n\n'), additional_kwargs={})])
  

In [29]:
retriever_response = retriever_chain.invoke(
    {
        "input": "What is Positional Encoding?"
    }
)

In [30]:
retriever_response

{'input': 'What is Positional Encoding?',
 'context': [Document(metadata={'source': 'https://medium.com/@asimsultan2/understanding-transformers-a-simplified-guide-with-easy-to-understand-examples-227fd8d31848', 'title': 'Understanding Transformers: A Simplified Guide with Easy-to-Understand Examples | by Asimsultan (Head of AI) | Medium', 'description': 'The Transformer architecture is a powerful model in natural language processing (NLP) that has revolutionized how machines understand and generate human language. However, the concepts behind…', 'language': 'en'}, page_content='how “quick” describes “fox,” while another might focus on the relationship between “jumps” and “over.”Positional EncodingTransformers process all words in a sentence simultaneously, so they need a way to understand the order of words — this is where positional encoding comes in. Positional encoding adds a unique identifier to each word based on its position in the sentence.Example: The “Classroom Roll Call” Anal

In [32]:
print(retriever_response['answer'])

1. What is the purpose of positional encoding in a Transformer model?

   Positional encoding is used to help the Transformer model understand the order of words in a sentence. Since Transformers process all words simultaneously, positional encoding adds a unique identifier to each word based on its position in the sentence, allowing the model to differentiate between words and their relationships based on their position.

2. How does positional encoding help in differentiating between words with the same meaning but in different positions?

   Positional encoding helps differentiate words with the same meaning but in different positions by providing a unique positional identifier to each word. This allows the model to understand how the same words relate differently in sentences with different structures, like "The cat sat on the mat" and "On the mat sat the cat."

3. Describe the tokenization process in the context of a Transformer model.

   In a Transformer model, tokenization is t